Association rules is a data mining technique. It analyzes user transaction data to discover which groups of products tend to be purchased together. 

To avoid memory issues, I draw 5% samples from all orders, that's  144 thousand orders in total over 37 thousand products for this analysis. 

In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
from tqdm import tqdm

pd.set_option('display.max_colwidth', None)

In [30]:
prior = pd.read_pickle('data/prior_order_details.pickle').set_index('order_id')
prior.head()

,user_id,order_number,days_since_prior_order,product_id,add_to_cart_order,reordered,_up_purchase_order,last_nth_order,user_max_order
order_id,,,,,,,,,
2539329,1,1,NaN,196,1,0,1.0,10,10
2539329,1,1,NaN,14084,2,0,1.0,10,10
2539329,1,1,NaN,12427,3,0,1.0,10,10
2539329,1,1,NaN,26088,4,0,1.0,10,10
2539329,1,1,NaN,26405,5,0,1.0,10,10


In [31]:
prior.shape

(32434489, 9)

In [32]:
num_products_order = prior.groupby('order_id')['product_id'].nunique()
num_products_order.head(3)

order_id
2     9
3     8
4    13
Name: product_id, dtype: int64

In [33]:
num_products_order.describe()

count    3.214874e+06
mean     1.008888e+01
std      7.525398e+00
min      1.000000e+00
25%      5.000000e+00
50%      8.000000e+00
75%      1.400000e+01
max      1.450000e+02
Name: product_id, dtype: float64

The largest order has 145 products! This is way more products than the average basket size. To speed up association algorithm later, we'd better remove some outlier orders

In [34]:
from utils import max_no_outliers
max_cap = max_no_outliers(num_products_order)
max_cap

27.5

select orders with more than 2 different products and less than max allowed products

In [35]:
prior = prior.loc[num_products_order[(num_products_order>=2) & (num_products_order<=max_cap)].index]

Due to memory limitation, we need to sample on order level 

In [36]:
np.random.seed(0)
sample_frac = 0.05
unique_orderid = prior.index.unique()
num_orders = len(unique_orderid)
selected_orders = np.random.choice(unique_orderid, size=int(num_orders*sample_frac))

prior = prior.loc[selected_orders]
print(prior.index.nunique())
print(prior.shape)

144040
(1432450, 9)


Transform data into the one-hot format required by the associaiton rules algorithm. Each row is an order and each column is a product

In [37]:
df_oh = pd.DataFrame(index=prior.index.unique(), columns=prior['product_id'].unique(), dtype=bool)
df_oh = ~df_oh
df_oh.shape

(144040, 37869)

In [38]:
df_oh.head(3)

,41302,24382,12674,49622,4443,845,37354,10527,32133,45190,...,48502,19872,25572,25607,46985,16667,8294,44934,26051,26833
order_id,,,,,,,,,,,,,,,,,,,,,
354409,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1720541,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2566718,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [39]:
for i, j in tqdm(prior['product_id'].reset_index().values):
    df_oh.loc[i, j] = True

100%|██████████████████████████████| 1432450/1432450 [01:08<00:00, 20953.58it/s]


Get frequent itemsets
- `min_support` = 0.001, requires the itemsets have to appear more than $0.001 \times 144,000= 144$ times in the transactions

In [40]:
frequent_itemsets = fpgrowth(df_oh, min_support = 0.001, use_colnames=True)
print(frequent_itemsets.shape)

(3312, 2)


Get association rules
- `metric`='confidence' and `min_threshold` requires the rules has to 5% confidence

In [41]:
rules = association_rules(frequent_itemsets, metric='confidence'
                          , min_threshold=0.05, support_only=False)[['antecedents', 'consequents', 'antecedent support',
                                                                    'consequent support', 'support', 'confidence', 'lift']]
rules.shape

(2175, 7)

In [42]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
0,(5785),(13176),0.01526,0.116364,0.002964,0.194268,1.669488
1,(5785),(24852),0.01526,0.148459,0.002895,0.189718,1.277917
2,(5785),(21137),0.01526,0.081248,0.002354,0.154231,1.898270
3,(5785),(27966),0.01526,0.041155,0.001569,0.102821,2.498364
4,(5785),(47209),0.01526,0.064010,0.001729,0.113285,1.769799


change ids in frozen set to readable names

In [43]:
def get_products_names(pids):
    products = pd.read_pickle('data/products.pickle')[['product_id', 'product_name']].set_index('product_id')
    names = []
    for pid in pids:
        names.append(products.at[pid, 'product_name'])
    return names

def get_departments(pids):
    products = pd.read_pickle('data/products.pickle')[['product_id', 'department']].set_index('product_id')
    departments = []
    for pid in pids:
        departments.append(products.at[pid, 'department'])
    return departments

def get_aisles(pids):
    products = pd.read_pickle('data/products.pickle')[['product_id', 'aisle']].set_index('product_id')
    aisles = []
    for pid in pids:
        aisles.append(products.at[pid, 'aisle'])
    return aisles

In [44]:
rules.rename(columns={'antecedents':'antecedent_pid',
                     'consequents':'consequent_pid'}, inplace=True)

In [45]:
rules['antecedents'] = rules['antecedent_pid'].apply(lambda x: get_products_names(x))
rules['consequents'] = rules['consequent_pid'].apply(lambda x: get_products_names(x))

rules['antecedent_department'] = rules['antecedent_pid'].apply(lambda x: get_departments(x))
rules['consequent_department'] = rules['consequent_pid'].apply(lambda x: get_departments(x))

Sort the results by **lift** and filter rules with more than 40% **confidence**

Three Key Metrics to Evaluate Association Rules:
1. Support \
It measures the frequency of an itemset <antecedents, consequents> in the transaction. The more frequent is the itemset, the more beneficial when we make use the rules related to this itemset. 
2. Confidence \
Confidence is a conditional probability. It's one of the methods of measuring how strong the rules is. It answers question of "out of all the people who purchase antecedent prdocts, what's the purcentage of people who also purchase consequent products". \
Confidence can be deceptive if the antecedents or consequents has a high level of support, we can have a high value of confidence even when the purchases of antecedents and consequents are independent. 
$$\text{Confidence(antecedents, consequents)} = \dfrac{\text{support(antecedents, consequents)}}{\text{support(antecedents)}}$$
3. Lift \
To complement Confidence, Lift is another way to judge strength of an assocation rule. Lift measure how dependent the antecedents and consequents are. If they are very dependent, the rule is strong and vice versa. 
$$\text{Lift(antecedents, consequents)} = \dfrac{\text{support(antecedents, consequents)}}{\text{support(confidence)} \times \text{support(consequents)}}$$

    - If purchases of antecedents and consequents are independent, lift(antecedents, consequents) = 1 because of independent of probabilities
    - If have a positive relationship, lift(antecedents, consequents) > 1 
    - If have a negative relationship, lift(antecedents, consequents) < 1

In [46]:
rules.sort_values(by='lift', ascending=False, inplace=True)
rules = rules.loc[rules.confidence >= 0.4]
rules

,antecedent_pid,consequent_pid,antecedent support,consequent support,support,confidence,lift,antecedents,consequents,antecedent_department,consequent_department
1791,(36865),(28465),0.004881,0.005623,0.002048,0.419630,74.621639,[Non Fat Raspberry Yogurt],[Icelandic Style Skyr Blueberry Non-fat Yogurt],[dairy eggs],[dairy eggs]
2130,(4957),(33754),0.005686,0.008873,0.002527,0.444444,50.092158,[Total 2% Lowfat Greek Strained Yogurt With Blueberry],[Total 2% with Strawberry Lowfat Greek Strained Yogurt],[dairy eggs],[dairy eggs]
1852,(33787),(33754),0.005575,0.008873,0.002270,0.407223,45.897018,[Total 2% Lowfat Greek Strained Yogurt with Peach],[Total 2% with Strawberry Lowfat Greek Strained Yogurt],[dairy eggs],[dairy eggs]
1467,(46149),(196),0.002520,0.010921,0.001229,0.487603,44.649956,[Zero Calorie Cola],[Soda],[beverages],[beverages]
727,"(44632, 21709)",(35221),0.002631,0.013698,0.001312,0.498681,36.406474,"[Sparkling Water Grapefruit, Sparkling Lemon Water]",[Lime Sparkling Water],"[beverages, beverages]",[beverages]
729,"(21709, 35221)",(44632),0.002492,0.023119,0.001312,0.526462,22.772265,"[Sparkling Lemon Water, Lime Sparkling Water]",[Sparkling Water Grapefruit],"[beverages, beverages]",[beverages]
1314,(41065),(45007),0.003319,0.031505,0.001486,0.447699,14.210341,[Organic Yellow Squash],[Organic Zucchini],[produce],[produce]
37,"(47209, 39275)",(13176),0.003464,0.116364,0.001458,0.420842,3.616612,"[Organic Hass Avocado, Organic Blueberries]",[Bag of Organic Bananas],"[produce, produce]",[produce]
541,"(39928, 47209)",(13176),0.002451,0.116364,0.001021,0.416431,3.578704,"[Organic Kiwi, Organic Hass Avocado]",[Bag of Organic Bananas],"[produce, produce]",[produce]
418,"(47209, 27966)",(13176),0.006956,0.116364,0.002846,0.409182,3.516409,"[Organic Hass Avocado, Organic Raspberries]",[Bag of Organic Bananas],"[produce, produce]",[produce]


What tends to be bought together?
- the same type of products but with different flavors, such as Raspberry Yogurt and Blueberry Yogurt, Grapefruit Sparkling Water, and Lime Sparking Water.
- products that could be used in combination, such as Yellow Squash and Zucchini (an easy sautee recipe), banana, avocado, and berries (common ingredients for smoothies)

It would be a good idea to create bundle discounts on them or set up reminder signs for customers

In [47]:
rules[['antecedent_pid', 'consequent_pid', 'antecedents', 'consequents', 'antecedent support', 'consequent support',
      'support', 'confidence', 'lift']].to_pickle('data/product_association_rules.pickle')